## Deliverable 2. Create a Customer Travel Destinations Map.

In [28]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [29]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hermanus,ZA,2020-09-04 18:36:38,-34.42,19.23,57.00,80,1,13.42
1,1,Saint-Augustin,CA,2020-09-04 18:36:38,51.23,-58.65,63.52,93,100,19.80
2,2,Pevek,RU,2020-09-04 18:36:38,69.70,170.31,34.75,88,86,4.61
3,3,Wonthaggi,AU,2020-09-04 18:36:38,-38.60,145.59,52.00,87,10,15.64
4,4,Provideniya,RU,2020-09-04 18:36:39,64.38,-173.30,41.05,69,10,3.22


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp =float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Saint-Augustin,CA,2020-09-04 18:36:38,51.23,-58.65,63.52,93,100,19.80
7,7,Nicoya,CR,2020-09-04 18:36:39,10.15,-85.45,87.80,66,75,6.93
8,8,Mkokotoni,TZ,2020-09-04 18:36:39,-5.88,39.25,77.00,83,20,12.75
9,9,East London,ZA,2020-09-04 18:36:40,-33.02,27.91,66.20,82,0,11.41
10,10,Rikitea,PF,2020-09-04 18:36:40,-23.12,-134.97,72.25,82,17,14.16


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       380
City          380
Country       379
Date          380
Lat           380
Lng           380
Max Temp      380
Humidity      380
Cloudiness    380
Wind Speed    380
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
clean_df= preferred_cities_df
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Saint-Augustin,CA,2020-09-04 18:36:38,51.23,-58.65,63.52,93,100,19.80
7,7,Nicoya,CR,2020-09-04 18:36:39,10.15,-85.45,87.80,66,75,6.93
8,8,Mkokotoni,TZ,2020-09-04 18:36:39,-5.88,39.25,77.00,83,20,12.75
9,9,East London,ZA,2020-09-04 18:36:40,-33.02,27.91,66.20,82,0,11.41
10,10,Rikitea,PF,2020-09-04 18:36:40,-23.12,-134.97,72.25,82,17,14.16
...,...,...,...,...,...,...,...,...,...,...
556,556,Penarroya-Pueblonuevo,ES,2020-09-04 18:38:58,38.30,-5.27,93.20,21,0,3.36
557,557,Guaruja,BR,2020-09-04 18:38:59,-23.99,-46.26,72.00,88,90,5.82
558,558,Johnstown,US,2020-09-04 18:38:59,40.33,-78.92,73.00,53,1,14.99
559,559,Pangai,TO,2020-09-04 18:39:00,-19.80,-174.35,70.25,67,0,18.16


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Saint-Augustin,CA,63.52,51.23,-58.65,
7,Nicoya,CR,87.80,10.15,-85.45,
8,Mkokotoni,TZ,77.00,-5.88,39.25,
9,East London,ZA,66.20,-33.02,27.91,
10,Rikitea,PF,72.25,-23.12,-134.97,
11,Port Alfred,ZA,74.62,-33.59,26.89,
12,Vaini,TO,68.00,-21.20,-175.20,
15,Phek,IN,61.21,25.67,94.50,
16,Bethel,US,82.00,41.37,-73.41,
17,Mahebourg,MU,68.00,-20.41,57.70,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Saint-Augustin,CA,63.52,51.23,-58.65,
7,Nicoya,CR,87.80,10.15,-85.45,Guayacan Real
8,Mkokotoni,TZ,77.00,-5.88,39.25,
9,East London,ZA,66.20,-33.02,27.91,Tu Casa
10,Rikitea,PF,72.25,-23.12,-134.97,Pension Maro'i
...,...,...,...,...,...,...
556,Penarroya-Pueblonuevo,ES,93.20,38.30,-5.27,Casa Rural Los Miraderos De Cristina
557,Guaruja,BR,72.00,-23.99,-46.26,Charme Hotel Guarujá
558,Johnstown,US,73.00,40.33,-78.92,Holiday Inn Johnstown-Downtown
559,Pangai,TO,70.25,-19.80,-174.35,Ha'apai Beach Resort


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig= gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))